This notebook is associated with the paper "The relative class number one problem for function fields, III" by K.S. Kedlaya. It runs in SageMath (tested using version 9.5beta9) and depends on Magma (tested using version
2.26-9).

In this notebook, we identify trigonal curves of genus 6 and Maroni invariant 0 which are candidates for the curve $C$ in a purely geometric quadratic extension $F'/F$ of relative class number 1. Allow 2 minutes for completion.

In [1]:
load("preamble.sage")

Construct the set of $\mathbb{F}_2$-rational points of $\mathbf{P}^1 \times \mathbf{P}^1$.

In [2]:
F = GF(2)
P.<x0,x1,y0,y1> = PolynomialRing(F, 4)

In [3]:
S1 = [vector(t) for t in ProjectiveSpace(F, 1)]
for v in S1:
    v.set_immutable()
S = list(itertools.product(S1, S1))
len(S)

9

Construct the group $\mathrm{GL}(2,\mathbb{F}_2) \times \mathrm{GL}(2, \mathbb{F}_2)$, presented as a subgroup of $\mathrm{GL}(4, \mathbb{F}_2)$.

In [4]:
l0 = [block_matrix(2,2,[g.matrix(),0,0,identity_matrix(2)], subdivide=False) for g in GL(2,F).gens()] +\
       [block_matrix(2,2,[identity_matrix(2),0,0,g.matrix()], subdivide=False) for g in GL(2,F).gens()]
G0 = GL(4,F).subgroup(l0)
G0.order()

36

Use an orbit lookup tree to find $\mathrm{GL}(2,\mathbb{F}_2) \times \mathrm{GL}(2, \mathbb{F}_2)$-orbit representatives for 6-tuples of $\mathbb{F}_2$-points in $\mathbf{P}^1 \times \mathbf{P}^1$.

In [5]:
def apply_group_elem(g, x):
    g1 = g.submatrix(nrows=2,ncols=2)
    g2 = g.submatrix(row=2,col=2)
    v1 = g1*x[0]
    v2 = g2*x[1]
    v1.set_immutable()
    v2.set_immutable()
    return (v1, v2)

In [6]:
def stabilizer(x):
    G1 = vec_stab(Matrix(F, x[0]), transpose=True)
    G2 = vec_stab(Matrix(F, x[1]), transpose=True)
    l0 = [block_matrix(2,2,[g.matrix(),0,0,identity_matrix(2)], subdivide=False) for g in G1.gens()] + \
        [block_matrix(2,2,[identity_matrix(2),0,0,g.matrix()], subdivide=False) for g in G2.gens()]
    return GL(4, F).subgroup(l0)

In [7]:
def optimized_rep(g):
    return g.matrix()

In [8]:
methods = {'apply_group_elem': apply_group_elem,
           'stabilizer': stabilizer,
           'optimized_rep': optimized_rep}
tree = build_orbit_tree(G0, S, 6, methods, verbose=False)

For each orbit representative of $k$-tuples with $k \in \{4,5,6\}$, find $(3,4)$-curves passing through precisely those points.

In [9]:
monos3 = [prod(x) for x in itertools.combinations_with_replacement([x0,x1],3)]
monos4 = [prod(x) for x in itertools.combinations_with_replacement([y0,y1],4)]
monos34 = [prod(x) for x in itertools.product(monos3, monos4)]
len(monos34)

20

In [10]:
coords34 = {x: vector(F, (mu(*x[0], *x[1]) for mu in monos34)) for x in S}

In [11]:
def vec_to_gen(vec):
    return sum(vec[i]*monos34[i] for i in range(20))

In [12]:
curves = defaultdict(list)
perp = Matrix([coords34[x] for x in S])
for i in range(4,7):
    for vecs in green_nodes(tree, i):
        target = vector(F, (0 if x in vecs else 1 for x in S))
        for w in solve_right_iterator(perp, target):
            curves[(i,)].append(vec_to_gen(w))

In [13]:
[(s, len(curves[s])) for s in curves]

[((4,), 14336), ((5,), 14336), ((6,), 12288)]

Enforce the desired point counts over $\mathbb{F}_{2^i}$ for $i=2,3,4$ using commutative algebra.

In [14]:
def count_by_ideal(gen, n):
    J = P.ideal([gen] + [y^(2^n) + y for y in P.gens()])
    return (J.vector_space_dimension() - 2^(2*n+1) + 1) // (2^n-1)^2

In [15]:
for n in range(2, 5):
    tmp = set(t[:n] for t in targets6)
    tmp2 = [s for s in curves if len(s) == n-1]
    for s in tmp2:
        for gen in curves[s]:
            i = count_by_ideal(gen, n)
            s1 = s + (i,)
            if s1 in tmp:
                curves[s1].append(gen)
        del curves[s]
    print([(s, len(curves[s])) for s in curves if len(s) == n])

[((4, 16), 20), ((4, 14), 326), ((5, 13), 1350), ((5, 11), 2970), ((5, 15), 322), ((6, 14), 720), ((6, 10), 3420)]
[((4, 16, 16), 5), ((4, 14, 16), 47), ((5, 13, 14), 264), ((5, 11, 11), 641), ((5, 15, 5), 12), ((6, 14, 12), 52), ((6, 14, 6), 23), ((6, 10, 9), 563)]
[((4, 14, 16, 18), 8), ((5, 13, 14, 25), 31), ((5, 11, 11, 31), 156), ((5, 11, 11, 39), 10), ((5, 15, 5, 35), 1), ((6, 14, 12, 26), 1), ((6, 14, 6, 34), 1), ((6, 14, 6, 26), 2), ((6, 10, 9, 38), 5)]


Close out this case.

In [16]:
I1 = P.ideal([x0,x1])
I2 = P.ideal([y0,y1])
CR = magma.CoxRing(P, [I1, I2], [[1,1,0,0],[0,0,1,1]], [])
proj = CR.ToricVariety()

In [17]:
closeout(curves, X=proj, genus=6)

Number of curves found: 20
Number of isomorphism classes found: 9
No covers found in this case!
Total time: 123 seconds
